In [1]:
import pandas as pd
import numpy as np
import itertools
import midi
import math
import random
from random import choice

In [2]:
fp = '/Users/10267068/Downloads/genome.txt'

In [3]:
len("""# This data file generated by 23andMe at: Fri Sep 28 10:52:40 2018
#
# This file contains raw genotype data, including data that is not used in 23andMe reports.
# This data has undergone a general quality review however only a subset of markers have been 
# individually validated for accuracy. As such, this data is suitable only for research, 
# educational, and informational use and not for medical or other use.
# 
# Below is a text version of your data.  Fields are TAB-separated
# Each line corresponds to a single SNP.  For each SNP, we provide its identifier 
# (an rsid or an internal id), its location on the reference human genome, and the 
# genotype call oriented with respect to the plus strand on the human reference sequence.
# We are using reference human assembly build 37 (also known as Annotation Release 104).
# Note that it is possible that data downloaded at different times may be different due to ongoing 
# improvements in our ability to call genotypes. More information about these changes can be found at:
# https://you.23andme.com/p/bd734743ae65a3ad/tools/data/download/
# 
# More information on reference human assembly build 37 (aka Annotation Release 104):
# http://www.ncbi.nlm.nih.gov/mapview/map_search.cgi?taxid=9606
#
# rsid	chromosome	position	genotype""".split('\n'))

20

In [4]:
snps_df = pd.read_csv(fp, header=19, delimiter='\t', low_memory=False)
snps_df.columns = ['rsid', 'chrom', 'pos', 'geno']
#snps_df = snps_df.set_index('rsid')
snps_df = snps_df[snps_df.geno != '--'].reset_index()
snps_df = snps_df.drop(columns='index')
snps_df = snps_df[:1000]

In [5]:
snps_df.head()

,rsid,chrom,pos,geno
0,rs548049170,1,69869,TT
1,rs9283150,1,565508,AA
2,rs116587930,1,727841,GG
3,rs3131972,1,752721,GG
4,rs12184325,1,754105,CC


In [6]:
# Create a MIDI file

In [7]:
def generate_pitch():
    pitch = choice(midi.NOTE_VALUES)
    return pitch
snps_df['pitch'] = snps_df.geno.map({
    snp: generate_pitch() for snp in snps_df.geno.unique()
})
# Instantiate a MIDI Pattern (contains a list of tracks)
pattern = midi.Pattern()
# Instantiate a MIDI Track (contains a list of MIDI events)
track = midi.Track()
# Append the track to the pattern
pattern.append(track)
for pitch in snps_df['pitch']:
    # Instantiate a MIDI note on event, append it to the track
    on = midi.NoteOnEvent(tick=0, velocity=100, pitch=pitch)
    track.append(on)
    # Instantiate a MIDI note off event, append it to the track
    off = midi.NoteOffEvent(tick=100, pitch=pitch)
    track.append(off)
# Add the end of track event, append it to the track
eot = midi.EndOfTrackEvent(tick=1)
track.append(eot)
# Save the pattern to disk
midi.write_midifile("/Users/10267068/Desktop/example.mid", pattern)

In [8]:
# generate the custom csv

In [9]:
pentatonic_scale = [
    440,  # A
    495,  # B
    556.875,  # C#
    660,  # E
    742.5,  # F
]
freq2midi = {
    440: 69,  # A
    495: 71,  # B
    556.875: 73,  # C#
    660: 76,  # E
    742.5: 78  # F
}
snps_df['freq'] = snps_df.geno.map({
    snp: choice(pentatonic_scale) for snp in snps_df.geno.unique()
})
snps_df['midi'] = snps_df.freq.map(freq2midi)
def length_gen(n):
    """generate a semi-random note length"""
    return int(3.5 \
               + math.sin(n/2) \
               + (2.5 * math.cos(n)) \
               + random.random())
snps_df['len'] = snps_df.index.map(length_gen)

In [10]:
snps_df.head()

,rsid,chrom,pos,geno,pitch,freq,midi,len
0,rs548049170,1,69869,TT,10,440.0,69,6
1,rs9283150,1,565508,AA,3,660.0,76,5
2,rs116587930,1,727841,GG,129,495.0,71,3
3,rs3131972,1,752721,GG,129,495.0,71,2
4,rs12184325,1,754105,CC,82,742.5,78,3


In [11]:
snps_df[['freq', 'midi', 'len']].to_clipboard()